In [1]:
import sys,os
import numpy as np

import keras
import pandas as pd
import re
import copy #can delete

import string
import matplotlib.pyplot as plt


from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding, Dropout
from keras.models import Model
from keras.models import Sequential
from keras.layers import LSTM, GRU
from keras.preprocessing.text import Tokenizer
from keras import optimizers
from keras import regularizers

from nltk.corpus import stopwords
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.porter import PorterStemmer

from sklearn.metrics import classification_report,accuracy_score,f1_score,precision_score,recall_score
from sklearn.cross_validation import cross_val_score,cross_val_predict,KFold,StratifiedKFold
from sklearn.metrics import precision_recall_fscore_support as score

np.random.seed(1)

Using Theano backend.
/usr/local/lib/python2.7/dist-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
MAX_SEQUENCE_LENGTH = 30 #max number of sentences in a message
MAX_NB_WORDS = 20000 #cap vocabulary
GLOVE_FILE = 'glove.twitter.27B/glove.twitter.27B.200d.txt'
EMBEDDING_DIM = 200 #size of word vector 
TWITTER_FILE = 'training-Obama-Romney-tweets.xlsx'
JAR_FILE = '/home/sreeraj/stanford-postagger-2016-10-31/stanford-postagger.jar'
MODEL_FILE = '/home/sreeraj/stanford-postagger-2016-10-31/models/english-left3words-distsim.tagger'
TOKENIZER = 'keras' #or use nltk
STEMMER = 'wordnet'


def get_Ytrue_Ypred(model,x,y):
    #Y matrix is [1,0,0] for class 0, [0,1,0] for class 1, [0,0,1] for class -1
    convert_to_label ={0:0,1:1,2:-1}
    model_predictions = model.predict(x)
    y_pred = np.zeros(len(y))
    y_true = np.zeros(len(y))
    #errors = 0.0
    for i in range(len(y)):
        y_pred[i] = convert_to_label[np.argmax(model_predictions[i])]
        y_true[i] = convert_to_label[np.argmax(y[i])]
        #if y_true[i] != y_pred[i]:
            #errors+=1.0
    return y_true,y_pred
    

# read the data
obama_data = pd.read_excel(TWITTER_FILE,names = ['date','time','text','sentiment'],parse_cols = 4,sheetname = 'Obama')
romney_data = pd.read_excel(TWITTER_FILE,names = ['date','time','text','sentiment'],parse_cols = 4,sheetname = 'Romney')

def get_data(data):
    """ get and clean the data """
    data = data.iloc[1:]
    data['text'] = data['text'].values.astype('unicode')
    data['date'] = data['date'].values.astype('str')
    data['time'] = data['time'].values.astype('unicode')
    # remove rows with mixed sentiment
    data = data[data['sentiment'] < 2]
    data.index = range(len(data))
    
    return data

obama_data = get_data(obama_data)
romney_data = get_data(romney_data)

obama_dataO = copy.deepcopy(obama_data)


/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-vers

In [3]:
emoticon_dictionary = {':)':' smileyface ','(:':' smileyface ','XD': ' happyface ',':D': ' smileyface ','>.<':' smileyface ',':-)':' smileyface ',';)':' winkface ',';D':' winkface ',':\'(':' cryingface '}

emoticons = [':\)','\(:','XD',':D','>\.<',':-\)',';\)',';D',':\'\(']

emoticon_pattern = re.compile(r'(' + '\s*|\s*'.join(emoticons) + r')')

# convert emoticons to words
def emoticon_converter(x):
    x = emoticon_pattern.sub(lambda i : emoticon_dictionary[i.group().replace(' ','')],x)   
    return x

obama_data['text'] = obama_data['text'].apply(emoticon_converter)
romney_data['text'] = romney_data['text'].apply(emoticon_converter)

# http://stackoverflow.com/questions/8870261/how-to-split-text-without-spaces-into-list-of-words
# convert hashtags into words
def separate_hashtag(x):
    for i in range(0,len(x)):
        hashtags = re.findall(r"#(\w+)", x[i])
        for words in hashtags:
            x[i] = re.sub('#'+ words,split_hashtag(words.lower()),x[i])
    return x

#obama_data['text'] = separate_hashtag(obama_data['text'])
#romney_data['text'] = separate_hashtag(romney_data['text'])




# remove punctuations
punc = ['\:','\;','\?','\$','\.','\(','\)','\#',',','-','\<','\>','\%','\*']
cond_1 = re.compile('|'.join(punc))
# remove tags
tags = ['<a>','</a>','<e>','</e>']
cond_2 = re.compile("|".join(tags))

def preprocess(data):
    """ preprocess the data"""
    # remove users
    data = data.apply(lambda x : re.sub(r'\@\s?\w+','',x))
    # remove hypertext 
    data = data.apply(lambda x : re.sub(r"http\S+",'',x)) #edited 
    # remove tags
    data = data.apply(lambda x : re.sub(cond_2,'',x))
    # remove punctuations
    data = data.apply(lambda x : re.sub(cond_1,'',x))
    # remove digits
    data = data.apply(lambda x : re.sub(r'[0-9]+','',x))
    # convert to ascii
    data = data.apply(lambda x: x.encode('utf-8')) #sometimes doesn't work
    # in that case
    printable = set(string.printable)
    for i in range(len(data)):
        data[i] = filter(lambda x: x in printable, data[i])
    
    return data

obama_data['text'] = preprocess(obama_data['text'])
romney_data['text'] = preprocess(romney_data['text'])


In [4]:
manual_stopwords_list = ['RT','MT']


stopwords_list = stopwords.words('english') + manual_stopwords_list

# stemming
class Tokenizer(object):
    def __init__(self,stemmer='porter'):
        self.stemmer = stemmer
        if stemmer == 'wordnet':
            self.wnl = WordNetLemmatizer()
        if stemmer == 'porter':
            self.wnl = PorterStemmer()
        if stemmer == 'snowball':
            self.wnl = SnowballStemmer('english')
    def __call__(self, doc):
        if self.stemmer == 'wordnet':
            return [self.wnl.lemmatize(t) for t in word_tokenize(doc)]
        else:
            return [self.wnl.stem(t) for t in word_tokenize(doc)]
                
            
def get_X_y(data):
    return data['text'],data['sentiment'].astype(int)


texts = romney_data['text']
labels = np.array(romney_data['sentiment'])

tokenizer = keras.preprocessing.text.Tokenizer(nb_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts) #list of lists, basically replaces each word with number

tokens = []
myTokenizer = Tokenizer(STEMMER)
for i in range(0,len(texts)):
    try:
        tokens.append(myTokenizer.__call__(texts[i]))
    except UnicodeDecodeError:
        pass
word_dict = {}
winx = 1
mysequences = []
tsq = []
for i in range(0,len(tokens)):
    for token in tokens[i]:
        if token not in word_dict:
            word_dict[token] = winx
            winx += 1
        tsq.append(word_dict[token])
    mysequences.append(tsq)
    tsq = []


    
word_index = tokenizer.word_index #key = word, value = number
#word_index = word_dict
#sequences = mysequences
if TOKENIZER == 'nltk':
    word_index = word_dict
    sequences = mysequences
# print('Found %s unique tokens.' % len(word_index))

#pad the data 
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
# print labels[0:4]
Y = labels
labels = keras.utils.np_utils.to_categorical(labels,nb_classes=3)
# print labels[0:4]

# print('Shape of data tensor:', data.shape)
# print('Shape of label tensor:', labels.shape)

In [5]:
embeddings_index = {}
f = open(GLOVE_FILE)
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

#prepare embedding matrix

#num_words = min(MAX_NB_WORDS, len(word_index))
num_words = len(word_index)+1
embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector


#create the embedding layer

Found 1193514 word vectors.


In [11]:
# create the model
np.random.seed(1)

def build_model():
    np.random.seed(1)
    l2 = regularizers.l2(0.01)
    l22 = regularizers.l2(0.01)
    model = Sequential()
    embedding_layer = Embedding(num_words,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=0)
    model.add(embedding_layer)
    #model.add(LSTM(10,return_sequences=True))
    model.add(GRU(150,return_sequences=False,dropout_W=0.6,dropout_U=0.5)) #can also use LSTM
    #model.add(LSTM(15,return_sequences=False,W_regularizer=l22))
    #model.add(Dropout(0.2))
#     model.add(Dense(30, activation='relu'))
    #model.add(Dropout(0.2))
    sgd = optimizers.SGD(lr=0.1, decay=1e-6, momentum=0.8, nesterov=True)
    model.add(Dense(len(labels[0]), activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
    return model


In [12]:
#k fold cross validaiton
avg_acc = []
avg_f1 = []
f_pos = []
f_neg = []
precision_pos = []
precision_neg = []
recall_pos = []
recall_neg = []

#data = data[0:500]
# kf = KFold(n=len(data),n_folds=10)
kf = StratifiedKFold(Y,n_folds=10)
for train,test in kf: #do the cross validation
    np.random.seed(1)
    x_train, x_val, y_train, y_val = data[train], data[test], labels[train], labels[test]
    
    model = build_model()
    model.fit(x_train, y_train, nb_epoch=15, batch_size=64,verbose=0) #ep = 20 .5979
    y_true,y_pred = get_Ytrue_Ypred(model,x_val,y_val)
    avg_acc.append(accuracy_score(y_true,y_pred))
    avg_f1.append(f1_score(y_true,y_pred,average='macro'))      
    print classification_report(y_true,y_pred)
    precision, recall, fscore, support = score(y_true, y_pred)
    f_pos.append(fscore[2])
    f_neg.append(fscore[0])
    precision_pos.append(precision[2])
    precision_neg.append(precision[0])
    recall_pos.append(recall[2])
    recall_neg.append(recall[0])
    


#print classification_report(y_true,y_pred)
print 'Average f1-score = ', np.mean(np.array(avg_f1))
print 'Overall Accuracy = ',100.0*np.mean(np.array(avg_acc)),'%'
print 'positive f1-score = ', np.mean(np.array(f_pos))
print 'negative f1-score = ', np.mean(np.array(f_neg))
print 'positive precision = ', np.mean(np.array(precision_pos))
print 'negative precision = ', np.mean(np.array(precision_neg))
print 'positive recall = ', np.mean(np.array(recall_pos))
print 'negative recall = ', np.mean(np.array(recall_neg))

             precision    recall  f1-score   support

       -1.0       0.61      0.95      0.74       290
        0.0       0.61      0.21      0.32       168
        1.0       0.72      0.35      0.47       108

avg / total       0.63      0.62      0.56       566

             precision    recall  f1-score   support

       -1.0       0.56      0.87      0.68       290
        0.0       0.44      0.14      0.22       168
        1.0       0.56      0.31      0.40       108

avg / total       0.52      0.55      0.49       566

             precision    recall  f1-score   support

       -1.0       0.58      0.92      0.71       290
        0.0       0.55      0.16      0.25       168
        1.0       0.53      0.26      0.35       108

avg / total       0.56      0.57      0.50       566

             precision    recall  f1-score   support

       -1.0       0.61      0.90      0.72       289
        0.0       0.54      0.27      0.36       168
        1.0       0.41      0.21    

In [ ]:
#
#
#get = word_index.items()
#for i in range(0,len(get)):
#    if get[i][1] == 6760:
#        print get[i][0]
#
#np.random.seed(1)
#model = build_model()
#history = model.fit(data, labels, nb_epoch=70, validation_split=0.2, shuffle=True)
#train_loss = history.history['loss']
#val_loss = history.history['val_loss']
#plt.plot(range(0,len(train_loss)),train_loss,label='train')
#plt.plot(range(0,len(train_loss)),val_loss,label='val')
#plt.legend(loc='left')
